In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Correccion temporal (22/mayo/2022)
#Tensorflow datasets tiene error al descargar el set de perros y gatos y lo solucionaron
#el 16 de mayo pero sigue fallando en los colabs. Entonces se agrega esta linea adicional
#Mas detalle aqui: https://github.com/tensorflow/datasets/issues/3918
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

#Descargar el set de datos de perros y gatos
datos, metadatos = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)

In [ ]:
#Imprimir los metadatos para revisarlos
metadatos

In [ ]:
#Una forma de mostrar 5 ejemplos del set
tfds.as_dataframe(datos['train'].take(5), metadatos)

In [ ]:
#Otra forma de mostrar ejemplos del set
tfds.show_examples(datos['train'], metadatos)

In [ ]:
#Manipular y visualizar el set
#Lo pasamos a TAMANO_IMG (100x100) y a blanco y negro (solo para visualizar)
import matplotlib.pyplot as plt
import cv2

#indico el tamapaño de las imagenes usando la funccion figure
plt.figure(figsize=(20,20))

#varuable de dimension que usaremos en la imagen
TAMANO_IMG=100

# se crea un indice para pedir que me muestre solo la imagen y la etiqueta
for i, (imagen, etiqueta) in enumerate(datos['train'].take(25)):

  #usamos cv2 para cambiar el tamaño de la imagen usando la funcion reize
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))

  #pasamos las imagenes a blanco y negro para que sea menos tardado
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

  #se usa la funcion subplot para que me muestre Nfila y Ncolumnas, e itere
  plt.subplot(5, 5, i+1)

  #ticks en arreglo vacio para que no me muestre cuadricula
  plt.xticks([])
  plt.yticks([])

  #mostramos las imagenes
  plt.imshow(imagen, cmap='gray')

In [9]:
#Variable que contendra todos los pares de los datos (imagen y etiqueta) ya modificados (blanco y negro, 100x100)
datos_entrenamiento = []

In [10]:
#iteramos en todos los datos de entenamiento
for i, (imagen, etiqueta) in enumerate(datos['train']):
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG, 1) #Cambiar tamano a 100,100,1
  datos_entrenamiento.append([imagen, etiqueta])

In [ ]:
#Ver los datos del primer indice (valores de los pixeles)
datos_entrenamiento[0]

In [ ]:
#Ver cuantos datos tengo en la variable
len(datos_entrenamiento)

In [13]:
#Preparar mis variables X (entradas) y Y (etiquetas) separadas

X = [] #imagenes de entrada (pixeles)
y = [] #etiquetas (perro o gato)

for imagen, etiqueta in datos_entrenamiento:
  X.append(imagen)
  y.append(etiqueta)

In [14]:
#Normalizar los datos de las X (imagenes). Se pasan a numero flotante y dividen entre 255 para quedar de 0-1 en lugar de 0-255
import numpy as np

X = np.array(X).astype(float) / 255

In [15]:
#Convertir etiquetas en arreglo simple entre 0 y 1
y = np.array(y)

In [ ]:
y

In [16]:
#Crear los modelos iniciales
#Usan sigmoid como salida (en lugar de softmax) para mostrar como podria funcionar con dicha funcion de activacion.
#Sigmoid regresa siempre datos entre 0 y 1. Realizamos el entrenamiento para al final considerar que si la respuesta se
#acerca a 0, es un gato, y si se acerca a 1, es un perro.

#primero modelo de datos. modelo denso
modeloDenso = tf.keras.models.Sequential([
  #capa de entrada recibiendo los pixeles
  tf.keras.layers.Flatten(input_shape=(100, 100, 1)),

  #dos capas de 150 neuronas cada una
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(150, activation='relu'),

  #capa de salida, dejamos 1 sola neurona de salida y utilizamos la activacion sigmoid que siempre devuelve un valor entre 0 y 1
  tf.keras.layers.Dense(1, activation='sigmoid')
])


#segundo modelo de datos usando una red neuronal convolucional
modeloCNN = tf.keras.models.Sequential([
  #se agregan 3 capas de capas convolucionales y de agrupacion maxima, pasando por 32,64 y 128 filtros
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  #se agraga una capa densa de 100 neuronas y la salida tambien con sigmoid
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
#Compilamos los modelos con el optimizador "adam". Usamos la funcion de perdida crossentropy binario ya que tenemos solo 2 opciones (perro o gato)
modeloDenso.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [18]:
#usamos esta libreria para poder usar TensorBoar
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
#La variable de tensorboard se envia en el arreglo de "callbacks"
#En este caso guarda datos en la carpeta indicada en cada epoca, de manera que despues
#Tensorboard los lee para hacer graficas

#creamos la variable TensonBoardDenso para estre entrenamiento del primer modelo
tensorboardDenso = TensorBoard(log_dir='logs/denso')
#entrenamos el modelo mandando la funcion fit y les envio la X(imagenes), y(etiquetas) y le indico un tamaño de lote
modeloDenso.fit(X, y, batch_size=32,
                #porcentaje para pruebas del 15%
                validation_split=0.15,
                #se usan 100 epocas
                epochs=100,
                #se agrega un arreglo de callback para guardar los resultados de cada epoca
                callbacks=[tensorboardDenso])

In [ ]:
#entrenamiento del segundo modelo
tensorboardCNN = TensorBoard(log_dir='logs/cnn')
modeloCNN.fit(X, y, batch_size=32,
                validation_split=0.15,
                epochs=100,
                callbacks=[tensorboardCNN])

In [ ]:
#ver las imagenes de la variable X sin modificaciones por aumento de datos
plt.figure(figsize=(20, 8))
for i in range(10):
  plt.subplot(2, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(X[i].reshape(100, 100), cmap="gray")

In [ ]:
#Realizar el aumento de datos con varias transformaciones. Al final, graficar 10 como ejemplo
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,#rotar la imagen de manera aleatoria
    width_shift_range=0.2,#mueve la imagen a los los lados
    height_shift_range=0.2,#mueve la imagen arriba y abajo
    shear_range=15,#se inclina la imagen
    zoom_range=[0.7, 1.4],#que tanto se puede hacer el acercamiento a la imagen
    horizontal_flip=True,#rotaciones aleatorias horizontal o vertual
    vertical_flip=True
)

datagen.fit(X)

plt.figure(figsize=(20,8))

for imagen, etiqueta in datagen.flow(X, y, batch_size=10, shuffle=False):
  for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imagen[i].reshape(100, 100), cmap="gray")
  break

In [27]:
modeloDenso_AD = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(150, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN_AD = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
modeloDenso_AD.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

modeloCNN_AD.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [29]:
#Separar los datos de entrenamiento y los datos de pruebas en variables diferentes
#datos de entrenamiento
len(X) * .85 #19700

#datos para pruebas
len(X) - 19700 #3562

#imagenes
X_entrenamiento = X[:19700]#85% de entrenamiento
X_validacion = X[19700:]#15%para pruebas

#etiquetas
y_entrenamiento = y[:19700]
y_validacion = y[19700:]

In [30]:
#Usar la funcion flow del generador para crear un iterador que podamos enviar como entrenamiento a la funcion FIT del modelo
data_gen_entrenamiento = datagen.flow(X_entrenamiento, y_entrenamiento, batch_size=32)

In [31]:
#comando de python para librerar memoria ram
import gc
gc.collect()

19836

In [ ]:
#entrenamiento modelo 1 denso con aumento de datos
tensorboardDenso_AD = TensorBoard(log_dir='logs/denso_AD')

#funcion fit
modeloDenso_AD.fit(
    data_gen_entrenamiento,
    epochs=100, batch_size=32,
    validation_data=(X_validacion, y_validacion),
    steps_per_epoch=int(np.ceil(len(X_entrenamiento) / float(32))),
    validation_steps=int(np.ceil(len(X_validacion) / float(32))),
    callbacks=[tensorboardDenso_AD]
)

In [ ]:
#entrenamiento modelo 2 convolucional con aumento de datos
tensorboardCNN_AD = TensorBoard(log_dir='logs-new/cnn_AD')

modeloCNN_AD.fit(
    data_gen_entrenamiento,
    epochs=150, batch_size=32,
    validation_data=(X_validacion, y_validacion),
    steps_per_epoch=int(np.ceil(len(X_entrenamiento) / float(32))),
    validation_steps=int(np.ceil(len(X_validacion) / float(32))),
    callbacks=[tensorboardCNN_AD]
)

In [35]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs

In [37]:
#se guarda el modelo que mejor resultado obtuvo y le indico el nombre
modeloCNN_AD.save('perros-gatos-cnn-ad.h5')

In [38]:
#instalamos tensorfliwjs con pip
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-decision-forests to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━

In [ ]:
#se crea una capeta para guardar el modelo
!mkdir carpeta_salida

In [ ]:
#exportamos el modelo H5 al formato de tensrflowjs para usarlo en un explorador
!tensorflowjs_converter --input_format keras perros-gatos-cnn-ad.h5 carpeta_salida